# Wind power estimation

## Import libraries

In [ ]:
import tempfile

import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
# Time
start = "1990-09"
stop = "2024-11"

# Locations
latitudes = [70.63]
longitudes = [22.46]
assert len(latitudes) == len(longitudes)

# Domains
carra_domains = ["east", "west"]

# Wind power data
wind_power_url = (
    "https://www.nve.no/media/16891/vindprod2002-2023_kraftverk_utcplus1.xlsx"
)

## Define request

In [ ]:
carra_id = "reanalysis-carra-height-levels"
carra_request = {
    "variable": ["wind_speed"],
    "height_level": ["75_m", "100_m"],
    "product_type": "analysis",
    "time": [f"{hour:02d}:00" for hour in range(0, 24, 3)],
    "data_format": "grib",
}

era5_id = "reanalysis-era5-single-levels"
era5_request = {
    "product_type": ["reanalysis"],
    "variable": ["100m_u_component_of_wind", "100m_v_component_of_wind"],
    "time": [f"{hour:02d}:00" for hour in range(24)],
    "area": [72, 20, 68, 24],
    "data_format": "grib",
    "download_format": "unarchived",
}

## Define functions to cache

In [ ]:
def preprocess(ds):
    (varname,) = set(ds.data_vars)
    return ds.rename({varname: "ws"}).expand_dims("height")


def find_nearest_locations(ds, latitudes, longitudes):
    ds = preprocess(ds)
    for coord in ("longitude", "latitude"):
        ds[f"nearest_{coord}"] = ds[coord]
        ds[f"nearest_{coord}"].attrs = {}
    grid = xr.Dataset(
        coords={
            "longitude": ("location", latitudes, {"standard_name": "longitude"}),
            "latitude": ("location", longitudes, {"standard_name": "latitude"}),
        }
    )
    return diagnostics.regrid(ds, grid, "nearest_s2d", locstream_out=True)


def compute_montly_mean(ds):
    ds = preprocess(ds)
    with tempfile.TemporaryDirectory(suffix=".zarr") as tmpdir:
        ds.chunk(x=10, y=10).to_zarr(tmpdir)
        ds = xr.open_dataset(tmpdir, chunks={})
        return (
            ds.resample(forecast_reference_time="1MS").mean(keep_attrs=True).compute()
        )

## Download and transform

In [ ]:
timeseries = []
maps = {}

carra_kwargs = {
    "collection_id": carra_id,
    "chunks": {"year": 1, "height_level": 1},
    "drop_variables": ["step"],
}
for domain in carra_domains:
    print(f"{domain = }")
    carra_kwargs["requests"] = download.update_request_date(
        carra_request | {"domain": f"{domain}_domain"},
        start,
        stop,
        stringify_dates=True,
    )
    ds_timeseries = download.download_and_transform(
        transform_func=find_nearest_locations,
        transform_func_kwargs={"latitudes": latitudes, "longitudes": longitudes},
        **carra_kwargs,
    )
    timeseries.append(ds_timeseries.expand_dims(domain=[domain]))

    ds_map = download.download_and_transform(
        transform_func=compute_montly_mean,
        **carra_kwargs,
    )
    maps[domain] = diagnostics.monthly_weighted_mean(ds_map).mean(
        "month", keep_attrs=True
    )

ds_timeseries = xr.concat(timeseries, "domain")

## Plot timeseries

In [ ]:
ds_timeseries["ws"].squeeze("location").plot(col="domain", row="height")

## Plot maps

In [ ]:
for domain, ds in maps.items():
    # This may take a few seconds as climatology is computed on the fly from monthly means
    plot.projected_map(ds["ws"], col="height")
    plt.suptitle(f"{domain = }")
    plt.show()